In [1]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [3]:
df = pd.read_excel('Mapping of tiers of colleges.xlsx')
df

,Institute ID,Name,City,State,Score,Rank,Type,Tier,Link
0,IR-O-U-0220,Indian Institute of Science,Bengaluru,Karnataka,83.16,1,University,1,https://www.nirfindia.org/2023/UniversityRanki...
1,IR-O-U-0109,Jawaharlal Nehru University,New Delhi,Delhi,68.92,2,University,1,https://www.nirfindia.org/2023/UniversityRanki...
2,IR-O-U-0108,Jamia Millia Islamia,New Delhi,Delhi,67.73,3,University,1,https://www.nirfindia.org/2023/UniversityRanki...
3,IR-O-U-0575,Jadavpur University,Kolkata,West Bengal,66.07,4,University,1,https://www.nirfindia.org/2023/UniversityRanki...
4,IR-O-U-0500,Banaras Hindu University,Varanasi,Uttar Pradesh,65.85,5,University,1,https://www.nirfindia.org/2023/UniversityRanki...
...,...,...,...,...,...,...,...,...,...
2410,NaN,"College of Computer Sciences, Pune",NaN,NaN,NaN,316,MSC,3,https://collegedunia.com/msc/nirf-ranked-colleges
2411,NaN,"AJ Aviation Academy, Bangalore",NaN,NaN,NaN,317,MSC,3,https://collegedunia.com/msc/nirf-ranked-colleges
2412,NaN,Lachoo Memorial College Science and Technology...,NaN,NaN,NaN,318,MSC,3,https://collegedunia.com/msc/nirf-ranked-colleges
2413,NaN,"Govindam Business School, New Delhi",NaN,NaN,NaN,319,MSC,3,https://collegedunia.com/msc/nirf-ranked-colleges


In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://www.nirfindia.org/2023/ManagementRanking.html'
response = requests.get(url)
page_content = response.text
soup = BeautifulSoup(page_content, "html.parser")

# Find the table(s) you want to extract based on HTML structure or attributes
# For example, you can use CSS selectors to locate specific tables
tables = soup.select("table")

In [5]:
df1 = pd.read_html(str(tables[0]))[0]
df1.columns

Index(['Institute ID', 'Name', 'City', 'State', 'Score', 'Rank'], dtype='object')

In [ ]:
df1.rename(columns={'Top BBA Colleges in India NIRF':'Name'},inplace=True)
df1

In [29]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
  options.add_argument('E:/Programs/geckodriver.exe')
  # options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

In [30]:
driver = driver_setup()

The version of firefox cannot be detected. Trying with latest driver version


In [199]:
url = 'https://collegedunia.com/msc/nirf-ranked-colleges'
driver.get(url)

In [200]:
# driver.execute_script("window.scrollTo(0,document.body.scrollHeight / 2);")
# time.sleep(3)
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# time.sleep(3)
# driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 3/ 4);")
# time.sleep(3)
table_elements = driver.find_elements(By.TAG_NAME, 'table')

In [201]:
print(len(table_elements))

145


In [203]:
table_elements = driver.find_elements(By.TAG_NAME, 'table')

# Initialize a list to store all the extracted data
all_data = []

# Loop through each table element
for table_element in table_elements:
    # Parse the HTML content of the table using BeautifulSoup
    table_html = table_element.get_attribute('outerHTML')
    soup = BeautifulSoup(table_html, 'html.parser')
    
    # Extract the desired elements from each row
    table_data = []
    for row in soup.find_all('tr'):
        row_data = row.find_all('td')
        if len(row_data) >= 2:
            rank = row_data[0].text.strip()
            
            # Check if the rank starts with "#"
            if rank.startswith("#"):
                # Check if an h3 element exists in the second column
                second_column = row_data[1]
                college_name = second_column.find('h3').text.strip() if second_column.find('h3') else "N/A"
                
                table_data.append((rank, college_name))
    
    # Append the table data to the list of all data
    all_data.append(table_data)

In [204]:
data = []
for  i  in all_data:
    for j in i:
        data.append(j)
df1 = pd.DataFrame(data,columns=['Rank','Name'])
df1

,Rank,Name
0,#1,"St Stephen's College, New Delhi"
1,#2,"Hindu College, New Delhi"
2,#3,"Lady Shri Ram College for Women - [LSR], New D..."
3,#4,"Miranda House, New Delhi"
4,#5,"Loyola College, Chennai"
...,...,...
315,#316,"College of Computer Sciences, Pune"
316,#317,"AJ Aviation Academy, Bangalore"
317,#318,Lachoo Memorial College Science and Technology...
318,#319,"Govindam Business School, New Delhi"


In [205]:
df1['Rank'] = df1['Rank'].str.replace('#','')
df1 

,Rank,Name
0,1,"St Stephen's College, New Delhi"
1,2,"Hindu College, New Delhi"
2,3,"Lady Shri Ram College for Women - [LSR], New D..."
3,4,"Miranda House, New Delhi"
4,5,"Loyola College, Chennai"
...,...,...
315,316,"College of Computer Sciences, Pune"
316,317,"AJ Aviation Academy, Bangalore"
317,318,Lachoo Memorial College Science and Technology...
318,319,"Govindam Business School, New Delhi"


In [6]:
df = pd.concat([df, df1], ignore_index=True)

In [7]:
df

,Institute ID,Name,City,State,Score,Rank,Type,Tier,Link
0,IR-O-U-0220,Indian Institute of Science,Bengaluru,Karnataka,83.16,1,University,1.0,https://www.nirfindia.org/2023/UniversityRanki...
1,IR-O-U-0109,Jawaharlal Nehru University,New Delhi,Delhi,68.92,2,University,1.0,https://www.nirfindia.org/2023/UniversityRanki...
2,IR-O-U-0108,Jamia Millia Islamia,New Delhi,Delhi,67.73,3,University,1.0,https://www.nirfindia.org/2023/UniversityRanki...
3,IR-O-U-0575,Jadavpur University,Kolkata,West Bengal,66.07,4,University,1.0,https://www.nirfindia.org/2023/UniversityRanki...
4,IR-O-U-0500,Banaras Hindu University,Varanasi,Uttar Pradesh,65.85,5,University,1.0,https://www.nirfindia.org/2023/UniversityRanki...
...,...,...,...,...,...,...,...,...,...
2510,IR-M-U-0107,Jamia HamdardMore Details |,New Delhi,Delhi,43.74,96,NaN,NaN,NaN
2511,IR-M-S-14553,National Institute of Agricultural Extension M...,Hyderabad,Telangana,43.47,97,NaN,NaN,NaN
2512,IR-M-S-15911,Indian Institute of ManagementMore Details |,Sirmaur,Himachal Pradesh,43.26,98,NaN,NaN,NaN
2513,IR-M-C-32887,New Delhi Institute Of ManagementMore Details |,Delhi,Delhi,42.61,99,NaN,NaN,NaN


In [8]:
df.drop_duplicates(inplace= True)
df.to_excel('Mapping of tiers of colleges.xlsx',index = False)